In [1]:
# Interception Effectiveness Modeling Based on Wilkening (1998)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Download latest version
path = kagglehub.dataset_download("piterfm/massive-missile-attacks-on-ukraine")

print("Path to dataset files:", path)

# Load data
df = pd.read_csv(f"{path}/missile_attacks_daily.csv")

/home/eznolin/codes/projeto-final-data-science-2025-01/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/eznolin/.cache/kagglehub/datasets/piterfm/massive-missile-attacks-on-ukraine/versions/136


In [2]:

# Drop rows with missing key values
df = df.dropna(subset=['launched', 'destroyed'])

# Calculate observed kill probability (K_obs)
df['K_obs'] = df['destroyed'] / df['launched']

# Assume default tracking probability and number of interceptors
P_track = 0.90
n_interceptors = 2  # You can later test with multiple values

# Function to calculate theoretical K based on SSPK and n
def calculate_K(P_track, k, n):
    return P_track * (1 - (1 - k)**n)

# Invert the formula to estimate SSPK (k) from observed K
# K_obs = P_track * (1 - (1 - k)^n) => k = 1 - (1 - K_obs / P_track)^(1/n)
def estimate_sspk(K_obs, P_track, n):
    with np.errstate(invalid='ignore'):  # avoid warnings for nan
        return 1 - np.power(1 - K_obs / P_track, 1/n)

df['estimated_sspk'] = estimate_sspk(df['K_obs'], P_track, n_interceptors)

# Display sample results
print(df[['time_start', 'model', 'launched', 'destroyed', 'K_obs', 'estimated_sspk']].head())
df.to_excel("./data_outputs/data_with_estimated_sspk.xlsx", index=False)

         time_start             model  launched  destroyed     K_obs  \
0  2025-06-21 21:00    Shahed-136/131      47.0       18.0  0.382979   
1  2025-06-21 21:00  Iskander-M/KN-23       2.0        0.0  0.000000   
2  2025-06-21 21:00             C-300       1.0        0.0  0.000000   
3  2025-06-20 20:00    Shahed-136/131     272.0      140.0  0.514706   
4  2025-06-21 02:40            Kalibr       4.0        3.0  0.750000   

   estimated_sspk  
0        0.242063  
1        0.000000  
2        0.000000  
3        0.345703  
4        0.591752  


In [ ]:
# Agrupar por 'model' e somar 'launched' e 'destroyed'
df_grouped = df.groupby('model', as_index=False)[['launched', 'destroyed']].sum()
df_grouped.to_excel('./data_outputs/model_launched_destroyed.xlsx', index=False)

plt.figure(figsize=(12, 6))
sns.boxplot(x='model', y='destroyed', data=df)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:

# Plot distribution of estimated SSPK values
plt.figure(figsize=(8,5))
sns.histplot(df['estimated_sspk'].dropna().astype(float), kde=True, bins=20)
plt.title("Estimated SSPK Distribution (n=3, P(track)=0.95)")
plt.xlabel("Estimated SSPK")
plt.ylabel("Frequency")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# --- Feature 1: Curvas K vs n por tipo de míssil ---
def plot_K_vs_n_by_model(df, P_track, k_range=np.linspace(0.5, 0.95, 5), max_n=6):
    missile_models = df['model'].dropna().unique()
    n_values = np.arange(1, max_n+1)
    for model in missile_models:
        plt.figure(figsize=(8, 5))
        for k in k_range:
            K_values = [calculate_K(P_track, k, n) for n in n_values]
            plt.plot(n_values, K_values, label=f"SSPK={k:.2f}")
        plt.title(f"K vs n for missile model: {model}")
        plt.xlabel("Number of Interceptors (n)")
        plt.ylabel("Kill Probability (K)")
        plt.ylim(0, 1)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

plot_K_vs_n_by_model(df, P_track)

In [ ]:
# --- Feature 3: Modelo de Machine Learning para prever K_obs ---
# Pré-processamento simples: converter categorias e tratar missing
ml_df = df.copy()
ml_df = ml_df.dropna(subset=['model', 'target', 'launched'])
ml_df['model'] = ml_df['model'].astype('category').cat.codes
ml_df['carrier'] = ml_df['carrier'].astype('category').cat.codes if 'carrier' in ml_df.columns else 0
ml_df['target'] = ml_df['target'].astype('category').cat.codes  # Encode 'target' as category codes

features = ['model', 'target', 'launched']
X = ml_df[features]
y = ml_df['K_obs']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Previsões e avaliação
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"\nRandom Forest MSE na previsão de K_obs: {mse:.4f}")

# Visualização da importância das features
feat_importances = pd.Series(model.feature_importances_, index=features)
feat_importances.sort_values().plot(kind='barh', title='Importância das Variáveis')
plt.xlabel("Importância")
plt.tight_layout()
plt.show()

In [ ]:
# --- Parte 4: Modelo de Machine Learning com Gradient Boosting ---
ml_df = df.dropna(subset=['model', 'target', 'launched', 'destroyed'])
ml_df['model'] = ml_df['model'].astype('category').cat.codes
ml_df['target'] = ml_df['target'].astype('category').cat.codes

features = ['model', 'target', 'launched', 'destroyed']
X = ml_df[features]
y = ml_df['K_obs']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"MSE do Gradient Boosting Regressor: {mse:.4f}")

# Importância das features
importances = pd.Series(model.feature_importances_, index=features)
importances.sort_values().plot(kind='barh', title='Importância das Variáveis')
plt.xlabel("Importância")
plt.tight_layout()
plt.show()
